# Lab 1 (counts as 50% of HW 1): Data processing with pandas 

UIC CS 418, Spring 2019

This lab will count as *50%* of your first homework assignment. You should be able to complete most, if not all, of it during class on January 28th, 2019.

In this lab you will see examples of some commonly used data wrangling tools in Python. In particular, we aim to give you some familiarity with:

* Slicing data frames
* Filtering data
* Grouped counts
* Joining two tables
* NA/Null values

Students posting solutions to this homework assignment is a violation of the academic integrity policy, as stated in the course syllabus. However, you are encouraged to follow along and re-use the partial solutions that the TAs provide during the lab.

## Due Date

This assignment is due at 11:59pm on February 2, 2019. Instructions on how to submit it to Gradescope are given at the end of the notebook.

# Practice (30%)

## Setup

You need to execute each step, in order for the next ones to work. First, import necessary libraries:

In [1]:
import pandas as pd
import numpy as np

The code below produces the data frames used in the examples:

In [2]:
heroes = pd.DataFrame(
    data={'color': ['red', 'green', 'black', 
                    'blue', 'black', 'red'],
          'first_seen_on': ['a', 'a', 'f', 'a', 'a', 'f'],
          'first_season': [2, 1, 2, 3, 3, 1]},
    index=['flash', 'arrow', 'vibe', 
           'atom', 'canary', 'firestorm']
)

identities = pd.DataFrame(
    data={'ego': ['barry allen', 'oliver queen', 'cisco ramon',
                  'ray palmer', 'sara lance', 
                  'martin stein', 'ronnie raymond'],
          'alter-ego': ['flash', 'arrow', 'vibe', 'atom',
                        'canary', 'firestorm', 'firestorm']}
)

teams = pd.DataFrame(
    data={'team': ['flash', 'arrow', 'flash', 'legends', 
                   'flash', 'legends', 'arrow'],
          'hero': ['flash', 'arrow', 'vibe', 'atom', 
                   'killer frost', 'firestorm', 'speedy']})

## Pandas and Wrangling

For the examples that follow, we will be using a toy data set containing information about superheroes in the Arrowverse.  In the `first_seen_on` column, `a` stands for Archer and `f`, Flash.

In [3]:
heroes

,color,first_seen_on,first_season
flash,red,a,2
arrow,green,a,1
vibe,black,f,2
atom,blue,a,3
canary,black,a,3
firestorm,red,f,1


In [4]:
identities

,ego,alter-ego
0,barry allen,flash
1,oliver queen,arrow
2,cisco ramon,vibe
3,ray palmer,atom
4,sara lance,canary
5,martin stein,firestorm
6,ronnie raymond,firestorm


In [5]:
teams

,team,hero
0,flash,flash
1,arrow,arrow
2,flash,vibe
3,legends,atom
4,flash,killer frost
5,legends,firestorm
6,arrow,speedy


### Slice and Dice

#### Column selection by label
To select a column of a `DataFrame` by column label, the safest and fastest way is to use the `.loc` method. General usage looks like `frame.loc[rowname,colname]`. (Reminder that the colon `:` means "everything").  For example, if we want the `color` column of the `ex` data frame, we would use :

In [6]:
heroes.loc[:, 'color']

flash          red
arrow        green
vibe         black
atom          blue
canary       black
firestorm      red
Name: color, dtype: object

Selecting multiple columns is easy. You just need to supply a list of column names. Here we select the color and value columns:

In [7]:
heroes.loc[:, ['color', 'first_season','first_seen_on']]

,color,first_season,first_seen_on
flash,red,2,a
arrow,green,1,a
vibe,black,2,f
atom,blue,3,a
canary,black,3,a
firestorm,red,1,f


While .loc is invaluable when writing production code, it may be a little too verbose for interactive use. One recommended alternative is the [] method, which takes on the form frame['colname'].

In [8]:
heroes['first_seen_on']

flash        a
arrow        a
vibe         f
atom         a
canary       a
firestorm    f
Name: first_seen_on, dtype: object

#### Row Selection by Label

Similarly, if we want to select a row by its label, we can use the same .loc method.

In [9]:
heroes.loc[['flash', 'vibe'], :]

,color,first_seen_on,first_season
flash,red,a,2
vibe,black,f,2


If we want all the columns returned, we can, for brevity, drop the colon without issue.

In [10]:
heroes.loc[['flash', 'vibe']]

,color,first_seen_on,first_season
flash,red,a,2
vibe,black,f,2


#### General Selection by Label

More generally you can slice across both rows and columns at the same time.  For example:

In [11]:
heroes.loc['flash':'canary', :'first_seen_on']

,color,first_seen_on
flash,red,a
arrow,green,a
vibe,black,f
atom,blue,a
canary,black,a


#### Selection by Integer Index

If you want to select rows and columns by position, the Data Frame has an analogous `.iloc` method for integer indexing. Remember that Python indexing starts at 0.

In [12]:
heroes.iloc[:4,:2]

,color,first_seen_on
flash,red,a
arrow,green,a
vibe,black,f
atom,blue,a


### Filtering with boolean arrays

Filtering is the process of removing unwanted material.  In your quest for cleaner data, you will undoubtedly filter your data at some point: whether it be for clearing up cases with missing values, culling out fishy outliers, or analyzing subgroups of your data set.  For example, we may be interested in characters that debuted in season 3 of Archer.  Note that compound expressions have to be grouped with parentheses.

In [13]:
heroes[(heroes['first_season']==3) & (heroes['first_seen_on']=='a')]

,color,first_seen_on,first_season
atom,blue,a,3
canary,black,a,3


#### Problem Solving Strategy
We want to highlight the strategy for filtering to answer the question above:

* **Identify the variables of interest**
    * Interested in the debut: `first_season` and `first_seen_on`
* **Translate the question into statements one with True/False answers**
    * Did the hero debut on Archer? $\rightarrow$ The hero has `first_seen_on` equal to `a`
    * Did the hero debut in season 3? $\rightarrow$ The hero has `first_season` equal to `3`
* **Translate the statements into boolean statements**
    * The hero has `first_seen_on` equal to `a` $\rightarrow$ `hero['first_seen_on']=='a'`
    * The hero has `first_season` equal to `3` $\rightarrow$ `heroes['first_season']==3`
* **Use the boolean array to filter the data**

Note that compound expressions have to be grouped with parentheses.

For your reference, some commonly used comparison operators are given below.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
==   | a == b   | Does a equal b?
<=   | a <= b   | Is a less than or equal to b?
>=   | a >= b   | Is a greater than or equal to b?
<    | a < b    | Is a less than b?
&#62;    | a &#62; b    | Is a greater than b?
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

An often-used operation missing from the above table is a test-of-membership.  The `Series.isin(values)` method returns a boolean array denoting whether each element of `Series` is in `values`.  We can then use the array to subset our data frame. For example, if we wanted to see which rows of `heroes` had values in $\{1,3\}$, we would use:

In [14]:
heroes[heroes['first_season'].isin([1,3])]

,color,first_seen_on,first_season
arrow,green,a,1
atom,blue,a,3
canary,black,a,3
firestorm,red,f,1


Notice that in both examples above, the expression in the brackets evaluates to a boolean series.  The general strategy for filtering data frames, then, is to write an expression of the form `frame[logical statement]`.

### Counting Rows

To count the number of instances of a value in a `Series`, we can use the `value_counts` method.  Below we count the number of instances of each color.

In [15]:
heroes['color'].value_counts()

black    2
red      2
green    1
blue     1
Name: color, dtype: int64

A more sophisticated analysis might involve counting the number of instances a tuple appears.  Here we count $(color,value)$ tuples.

In [16]:
heroes.groupby(['color', 'first_season']).size()

color  first_season
black  2               1
       3               1
blue   3               1
green  1               1
red    1               1
       2               1
dtype: int64

This returns a series that has been multi-indexed.  We'll eschew this topic for now.  To get a data frame back, we'll use the `reset_index` method, which also allows us to simulataneously name the new column.

In [17]:
heroes.groupby(['color', 'first_season']).size().reset_index(name='count')

,color,first_season,count
0,black,2,1
1,black,3,1
2,blue,3,1
3,green,1,1
4,red,1,1
5,red,2,1


### Joining Tables on One Column

Suppose we have another table that classifies superheroes into their respective teams.  Note that `canary` is not in this data set and that `killer frost` and `speedy` are additions that aren't in the original `heroes` set.

For simplicity of the example, we'll convert the index of the `heroes` data frame into an explicit column called `hero`.  A careful examination of the [documentation](http://pandas.pydata.org/pandas-docs/version/0.19.1/generated/pandas.DataFrame.merge.html) will reveal that joining on a mixture of the index and columns is possible.

In [18]:
heroes['hero'] = heroes.index
heroes

,color,first_seen_on,first_season,hero
flash,red,a,2,flash
arrow,green,a,1,arrow
vibe,black,f,2,vibe
atom,blue,a,3,atom
canary,black,a,3,canary
firestorm,red,f,1,firestorm


#### Inner Join

The inner join below returns rows representing the heroes that appear in both data frames.

In [19]:
pd.merge(heroes, teams, how='inner', on='hero')

,color,first_seen_on,first_season,hero,team
0,red,a,2,flash,flash
1,green,a,1,arrow,arrow
2,black,f,2,vibe,flash
3,blue,a,3,atom,legends
4,red,f,1,firestorm,legends


#### Left and right join
The left join returns rows representing heroes in the `heroes` ("left") data frame, augmented by information found in the `teams` data frame.  Its counterpart, the right join, would return heroes in the `teams` data frame.  Note that the `team` for hero `canary` is an `NaN` value, representing missing data.

In [20]:
pd.merge(heroes, teams, how='left', on='hero')

,color,first_seen_on,first_season,hero,team
0,red,a,2,flash,flash
1,green,a,1,arrow,arrow
2,black,f,2,vibe,flash
3,blue,a,3,atom,legends
4,black,a,3,canary,NaN
5,red,f,1,firestorm,legends


#### Outer join

An outer join on `hero` will return all heroes found in both the left and right data frames.  Any missing values are filled in with `NaN`.

In [21]:
pd.merge(heroes, teams, how='outer', on='hero')

,color,first_seen_on,first_season,hero,team
0,red,a,2.0,flash,flash
1,green,a,1.0,arrow,arrow
2,black,f,2.0,vibe,flash
3,blue,a,3.0,atom,legends
4,black,a,3.0,canary,NaN
5,red,f,1.0,firestorm,legends
6,NaN,NaN,NaN,killer frost,flash
7,NaN,NaN,NaN,speedy,arrow


#### More than one match?

If the values in the columns to be matched don't uniquely identify a row, then a cartesian product is formed in the merge.  For example, notice that `firestorm` has two different egos, so information from `heroes` had to be duplicated in the merge, once for each ego.

In [22]:
pd.merge(heroes, identities, how='inner', 
         left_on='hero', right_on='alter-ego')

,color,first_seen_on,first_season,hero,ego,alter-ego
0,red,a,2,flash,barry allen,flash
1,green,a,1,arrow,oliver queen,arrow
2,black,f,2,vibe,cisco ramon,vibe
3,blue,a,3,atom,ray palmer,atom
4,black,a,3,canary,sara lance,canary
5,red,f,1,firestorm,martin stein,firestorm
6,red,f,1,firestorm,ronnie raymond,firestorm


### Missing Values

There are a multitude of reasons why a data set might have missing values.  The current implementation of Pandas uses the numpy NaN to represent these null values (older implementations even used `-inf` and `inf`).  Future versions of Pandas might implement a true `null` value---keep your eyes peeled for this in updates!  More information can be found [http://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html](http://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)

Because of the specialness of missing values, they merit their own set of tools.  For this lab, we will focus on detection.  For replacement, see the docs.

In [23]:
x = np.nan
y = pd.merge(heroes, teams, how='outer', on='hero')['first_season']
y

0    2.0
1    1.0
2    2.0
3    3.0
4    3.0
5    1.0
6    NaN
7    NaN
Name: first_season, dtype: float64

To check if a value is null, we use the `isnull()` method for series and data frames.  Alternatively, there is a `pd.isnull()` function as well.

In [24]:
x.isnull() # won't work since x is neither a series nor a data frame

AttributeError: 'float' object has no attribute 'isnull'

In [25]:
pd.isnull(x)

True

In [26]:
y.isnull()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
Name: first_season, dtype: bool

In [27]:
pd.isnull(y)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
Name: first_season, dtype: bool

Since filtering out missing data is such a common operation, Pandas also has conveniently included the analogous `notnull()` methods and function for improved human readability.

In [28]:
y.notnull()

0     True
1     True
2     True
3     True
4     True
5     True
6    False
7    False
Name: first_season, dtype: bool

In [29]:
y[y.notnull()]

0    2.0
1    1.0
2    2.0
3    3.0
4    3.0
5    1.0
Name: first_season, dtype: float64

# Questions (20%)

The practice problems below use the department of transportation's "On-Time" flight data for all flights originating from SFO or OAK in January 2016. Information about the airports and airlines are contained in the comma-delimited files `airports.dat` and `airlines.dat`, respectively.  Both were sourced from http://openflights.org/data.html.

Disclaimer: There is a more direct way of dealing with time data that is not presented in these problems.  This activity is merely an academic exercise.

In [30]:
flights = pd.read_csv("flights.dat", dtype={'sched_dep_time': 'f8', 'sched_arr_time': 'f8'})
# show the first few rows, by default 5
flights.head() 

,year,month,day,date,carrier,tailnum,flight,origin,destination,sched_dep_time,actual_dep_time,sched_arr_time,actual_arr_time
0,2016,1,1,2016-01-01,AA,N3FLAA,208,SFO,MIA,630.0,628.0,1458.0,1431.0
1,2016,1,2,2016-01-02,AA,N3APAA,208,SFO,MIA,600.0,553.0,1428.0,1401.0
2,2016,1,3,2016-01-03,AA,N3DNAA,208,SFO,MIA,630.0,626.0,1458.0,1431.0
3,2016,1,4,2016-01-04,AA,N3FGAA,208,SFO,MIA,630.0,626.0,1458.0,1444.0
4,2016,1,5,2016-01-05,AA,N3KUAA,208,SFO,MIA,640.0,632.0,1458.0,1439.0


In [31]:
airports_cols = [
    'openflights_id',
    'name',
    'city',
    'country',
    'iata',
    'icao',
    'latitude',
    'longitude',
    'altitude',
    'tz',
    'dst',
    'tz_olson',
    'type',
    'airport_dsource'
]

airports = pd.read_csv("airports.dat", names=airports_cols)
airports.head(3)

,openflights_id,name,city,country,iata,icao,latitude,longitude,altitude,tz,dst,tz_olson,type,airport_dsource
0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10.0,U,Pacific/Port_Moresby,NaN,NaN
1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.788700,20,10.0,U,Pacific/Port_Moresby,NaN,NaN
2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10.0,U,Pacific/Port_Moresby,NaN,NaN


#### Question 1
It looks like the departure and arrival in `flights` were read in as floating-point numbers.  Write two functions, `extract_hour` and `extract_mins` that converts military time to hours and minutes, respectively. Hint: You may want to use modular arithmetic and integer division. Keep in mind that the data has not been cleaned and you need to check whether the extracted values are valid. 

In [32]:
def extract_hour(time):
    """
    Extracts hour information from military time.
    
    Args: 
        time (float64): series of time given in military format.  
          Takes on values in 0.0-2359.0 due to float64 representation.
    
    Returns:
        array (float64): series of input dimension with hour information.  
          Should only take on integer values in 0-23
    """
    result = time//100
    return result

In [33]:
def extract_mins(time):
    """
    Extracts minute information from military time
    
    Args: 
        time (float64): series of time given in military format.  
          Takes on values in 0.0-2359.0 due to float64 representation.
    
    Returns:
        array (float64): series of input dimension with hour information.  
          Should only take on integer values in 0-59
    """
    result = time%100
    return result

In [34]:
### write code to test your functions here and execute it
#extract_hour(630.0)
#extract_hour(flights['sched_dep_time'])

In [35]:
def convert_to_minofday(time):
    """
    Converts military time to minute of day
    
    Args:
        time (float64): series of time given in military format.  
          Takes on values in 0.0-2359.0 due to float64 representation.
    
    Returns:
        array (float64): series of input dimension with minute of day
    
    Example: 1:03pm is converted to 783.0
    """
    hours_mins = extract_hour(time)*60
    mins = extract_mins(time)
    result = hours_mins+mins
    return result
    

In [36]:
#convert_to_minofday(1303.0)
#convert_to_minofday(flights['sched_dep_time'])

In [37]:
def calc_time_diff(x, y):
    """
    Calculates delay times y - x
    
    Args:
        x (float64): series of scheduled time given in military format.  
          Takes on values in 0.0-2359.0 due to float64 representation.
        y (float64): series of same dimensions giving actual time
    
    Returns:
        array (float64): series of input dimension with delay time
    """
    
    scheduled = convert_to_minofday(x)
    actual = convert_to_minofday(y)
    return actual-scheduled

In [38]:
### write code to test your functions here and execute it. 
### your printed results should show the values of the following two variables
delay = calc_time_diff(flights['sched_dep_time'],flights['actual_dep_time'])       
#print(delay)

#delayed15 = delay[delay>=15]
#print(delayed15)

#### Question 3

Using your answer from question 2, find the full name of every destination city with a flight from SFO or OAK that was delayed by 15 or more minutes.  The airport codes used in `flights` are IATA codes.  Sort the cities alphabetically. Make sure you remove duplicates. You may find `drop_duplicates` and `sort_values` helpful.

In [39]:
### write code to test your functions here and execute it. 
### your printed results should show the values of the following two variables
f1 = flights[(flights['origin'] == 'OAK') | (flights['origin'] == 'SFO') & (delay >=15)]
airports["destination"] = airports["iata"]
joinedoutput = pd.merge(airports, f1, how='inner', on='destination')
joinedoutput = joinedoutput.sort_values(by = ['city']).drop_duplicates(['city'])
delayed_airports = joinedoutput['name']
delayed_destinations = joinedoutput['city']

In [45]:
#delayed_airports
#delayed_destinations


    # Submission

You're almost done! 

After executing all commands and completing this notebook, save your *lab1.ipynb* as a pdf file and upload it to Gradescope under *Homework 1 - lab 1 (written)*. Make sure you check that your pdf file includes all parts of your solution. We recommend using the browser (not jupyter) for saving the pdf. For Chrome on a Mac, this is under *File->Print...->Open PDF in Preview* and when the PDF opens in Preview you can use *Save...* to save it. This part will be graded based on completion and it constitutes *30%* of HW 1.

Next, you need to copy the functions from Questions 1 and 2 into the corresponding functions in *lab1.py*. Place your files *lab1.py* and *lab1.ipynb* in a zip file and upload the zip file to Gradescope under *Homework 1 - lab 1 (code)*. This part constitutes *20%* of HW 1. In order to get full points for this part, you need to pass all test cases that we will run against your *lab1.py* (and not the notebook) on Gradescope. To check whether your code runs locally, run the four tests in *tests_sample* from your command line: 

In [46]:
(cs418env) elena-macbook:lab1 elena$ python run_tests_sample.py

SyntaxError: invalid syntax (<ipython-input-46-f6c5d4251541>, line 1)

You should see the following output:

In [47]:
..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK

SyntaxError: invalid syntax (<ipython-input-47-23bb5a858733>, line 1)

Feel free to add more tests that check all parts of your code.

You can submit to Gradescope as many times as you would like. We will only consider your last submission. If your last submission is after the deadline, the late homework policy applies.

After submitting the zip file, you should see the following on your screen:

<img src="correct.png" align="left" float="left"/>

This indicates that all four of the tests ran successfully on the server, and you're done!